#### Загружаем необходимые библиотеки

In [109]:
import pandas as pd
from datetime import datetime, timedelta
import os
from os import listdir
from os.path import isfile, join
import shutil

import warnings
warnings.filterwarnings('ignore')

from pandas.util.testing import assert_frame_equal


#### Шаг итерации в подсчете ежедневных продаж

In [116]:
step = 20

# Функция обработки конкретного магазина

In [114]:
def shop_processing(inventory, sell, supply, state):
    daily = pd.DataFrame(columns=['date','apple','pen'])
    steal = pd.DataFrame(columns=['date','apple','pen'])
    aggregation = pd.DataFrame(columns=['year','state','apple_sold','apple_stolen','pen_sold','pen_stolen'])

    current_date = sell.index[0]

    apple_sold_year = 0
    pen_sold_year = 0
    apple_stolen = 0
    pen_stolen = 0

    for j in range((pd.to_datetime(sell.index[sell.size-1]) - pd.to_datetime(sell.index[0])).days + 1):
        daily_sell_apple = 0
        daily_sell_pen = 0
        
        day_sells = sell.loc[current_date].reset_index().sku_num
        for i in range(day_sells.size - 1, -1 , -step): 
            if day_sells[i][6] != 'p':
                k = 1
                while (i + k < day_sells.size - 1) and (day_sells[i + k][6] != 'p'):
                    k+=1 
                daily_sell_pen = day_sells.size - (i + k)
                break

        daily_sell_apple = day_sells.size - daily_sell_pen
    
        apple_sold_year += daily_sell_apple
        pen_sold_year += daily_sell_pen
    
        cur_date =  datetime.strptime(current_date, '%Y-%m-%d')
    
        if (current_date == sell.index[0]):
            daily = daily.append({'date':current_date, 'apple': supply.loc[current_date].apple - daily_sell_apple, 
                                  'pen': supply.loc[current_date].pen - daily_sell_pen}, ignore_index=True)
            current_date = str((cur_date + timedelta(days=1)).date())
            continue  
    
        daily = daily.append({'date':current_date, 'apple': daily.loc[j-1].apple - daily_sell_apple, 
                              'pen': daily.loc[j-1].pen - daily_sell_pen}, ignore_index=True)    

    
        if ((current_date[8:10] == '01') or (current_date[8:10] == '15')):
            daily.loc[j].apple += supply.loc[current_date].apple
            daily.loc[j].pen += supply.loc[current_date].pen
    
        if cur_date.day >= 28:
            if ((cur_date + timedelta(days=1)).date()).month != cur_date.month:
                apple_stolen += daily.loc[j].apple - inventory.loc[current_date].apple
                pen_stolen += daily.loc[j].pen - inventory.loc[current_date].pen
            
                steal = steal.append({'date':current_date, 'apple': daily.loc[j].apple - inventory.loc[current_date].apple, 
                                      'pen': daily.loc[j].pen - inventory.loc[current_date].pen}, ignore_index=True)
                daily.loc[j].apple = inventory.loc[current_date].apple
                daily.loc[j].pen = inventory.loc[current_date].pen
    
        if (cur_date.day == 31) and (cur_date.month == 12):
            aggregation = aggregation.append({'year': cur_date.year, 'state': state, 'apple_sold': apple_sold_year,
                                              'apple_stolen': apple_stolen, 'pen_sold': pen_sold_year, 'pen_stolen': pen_stolen 
                                             }, ignore_index=True)
            apple_sold_year = 0
            pen_sold_year = 0
            apple_stolen = 0
            pen_stolen = 0
        
        current_date = str((cur_date + timedelta(days=1)).date())
    
    return daily, steal, aggregation

# Итерируемся по магазинам в input

In [123]:
%%time

path_file = %pwd

# Создаем папку, в которую будут записываться ответы
try:
    os.mkdir(path_file + "\\our_answer")
except OSError:
    shutil.rmtree(path_file + "\\our_answer")
    os.mkdir(path_file + "\\our_answer")
    
files = [f for f in listdir(path_file + "\\input") 
         if isfile(join(path_file + "\\input", f))]

for i in range(int (len(files) / 3)):
    inventory = pd.read_csv("input\\" + files[3 * i], index_col='date') 
    sell = pd.read_csv("input\\" + files[3 * i + 1], index_col='date') 
    supply = pd.read_csv("input\\" + files[3 * i + 2], index_col='date')
    state = files[3 * i][:2] #название штата

    daily, steal, aggregation = shop_processing(inventory, sell, supply, state)
    
    daily.to_csv(path_file + "\\our_answer\\" + files[3 * i][0:6] + "daily.csv", index = False)
    steal.to_csv(path_file + "\\our_answer\\" + files[3 * i][0:6] + "steal.csv", index = False)
    
    if (i == 0):
        aggregation_sum = aggregation
    else:
        aggregation_sum = pd.concat([aggregation_sum, aggregation]).groupby(['year','state'])['apple_sold', 
                                                                                              'apple_stolen', 
                                                                                              'pen_sold', 
                                                                                              'pen_stolen'].sum()
        aggregation_sum = aggregation_sum.reset_index()
        
aggregation_sum.to_csv(path_file + "\\our_answer\\aggregation.csv", index = False)

Wall time: 2min 22s


# Сверим получившиеся данные с готовыми ответами

In [112]:
path_file = %pwd
files1 = [f for f in listdir(path_file + "\\answer") 
         if isfile(join(path_file + "\\answer", f))]

files2 = [f for f in listdir(path_file + "\\our_answer") 
         if isfile(join(path_file + "\\our_answer", f))]

error = (int (len(files1) / 2))
for i in range(int (len(files1) / 2)):
    daily1 = pd.read_csv("answer\\" + files1[2 * i])
    steal1 = pd.read_csv("answer\\" + files1[2 * i + 1])
    
    daily2 = pd.read_csv("our_answer\\" + files2[2 * i + 1])
    steal2 = pd.read_csv("our_answer\\" + files2[2 * i + 2])
    
    try:
        assert_frame_equal(daily1, daily2)
        assert_frame_equal(steal1, steal2)
    except:
        if ((daily1.size > daily2.size) | (steal1.size > steal2.size)):
            print("There are differences")
            error-=1
if ((int (len(files1) / 2)) == error):
        print("EVERYTHING'S OKAY")


EVERYTHING'S OKAY


In [122]:
aggregation_sum.set_index("year")


,state,apple_sold,apple_stolen,pen_sold,pen_stolen
year,,,,,
2006,MS,2152006,418,155633,461
2007,MS,2150384,377,154730,346
2008,MS,2163559,383,154597,382
2009,MS,2152502,433,155409,454
2010,MS,2149787,418,155523,441
2011,MS,2154860,436,154158,452
2012,MS,2160040,381,155798,421
2013,MS,2157901,361,154496,444
2014,MS,2153434,433,154687,441
